In [1]:
from pathlib import Path
import napari
import tifffile
import numpy as np
from cellpose import models
import skimage
from skimage import measure
from scipy.ndimage import binary_erosion, binary_dilation, label
import pyclesperanto_prototype as cle
from apoc import ObjectSegmenter
import pandas as pd

In [2]:
directory_path = Path("./test_data/")
images = []

# Iterate through the lsm files in the directory
for file_path in directory_path.glob("*.lsm"):
    images.append(str(file_path))
    
images

['test_data\\DSB Iba1 20_40X_CA1_test.lsm',
 'test_data\\DSB Iba1 20_40X_DG_test.lsm',
 'test_data\\DSB Iba1 21_40X_DG_test2.lsm',
 'test_data\\DSB Iba1 33_40X_CA3.lsm',
 'test_data\\DSB Iba1 33_40X_DG.lsm',
 'test_data\\DSB Iba1 43_40X_CTX2.lsm',
 'test_data\\DSB Iba1 45_40X_CTX3.lsm',
 'test_data\\DSB Iba1 45_40X_DG.lsm',
 'test_data\\DSB Iba1 58_40X_CTX2.lsm',
 'test_data\\DSB Iba1 58_40X_CTX3.lsm']

In [3]:
# Function to count particles in nuclei
def count_particles_in_nuclei(labeled_particles, labeled_nuclei, num_nuclei):
    particle_counts = np.zeros(num_nuclei, dtype=int)
    for i in range(1, num_nuclei + 1):
        nuclei_mask = labeled_nuclei == i
        # Count the number of unique particles in the nucleus
        unique_particles = np.unique(labeled_particles[nuclei_mask])
        # Exclude the background label (0)
        particle_counts[i - 1] = len(unique_particles[unique_particles > 0])
    return particle_counts


In [4]:
viewer = napari.Viewer(ndisplay=2)
model = models.Cellpose(gpu=True, model_type='nuclei')
stats = []

for image in images:
    
    # Extract filename
    file_path = Path(image)
    filename = file_path.stem

    # Read the image file 
    img = tifffile.imread(image)

    # Extract the stack containing the nuclei (0), dna_damage (1) and microglia channel (2)
    nuclei_stack = img[:,0,:,:]
    dna_damage_stack = img[:,1,:,:]
    microglia_stack = img[:,2,:,:]

    # Perform maximum intensity projections
    nuclei_mip = np.max(nuclei_stack, axis=0)
    dna_damage_mip = np.max(dna_damage_stack, axis=0)
    microglia_mip = np.max(microglia_stack, axis=0)

    # Stack the arrays along a new axis (axis 0)
    stack = np.stack((nuclei_mip, dna_damage_mip, microglia_mip), axis=0)

    # Create a copy of nuclei_mip
    input_img = nuclei_mip.copy()

    # Might need to perform a Gaussian-blur before
    post_gaussian_img = skimage.filters.gaussian(input_img, sigma=6)

    # Predict nuclei nuclei_masks using cellpose
    nuclei_masks, flows, styles, diams = model.eval(post_gaussian_img, diameter=None, channels=[0, 0], net_avg=False)

    # Dilate nuclei to make sure the dna_damage_foci objects sit inside
    dilated_nuclei_masks = cle.dilate_labels(nuclei_masks, radius=3)
    # Erode dilated_nuclei_masks to obtain separate objects upon binarization
    eroded_nuclei_masks = cle.erode_labels(dilated_nuclei_masks, radius=1)

    # Show the resulting stack in Napari
    viewer.add_image(stack, name=f"{filename}_mip")
    #viewer.add_image(nuclei_mip, name=f"{filename}_nuclei")
    viewer.add_labels(nuclei_masks, name=f"{filename}_nuclei_mask")
    viewer.add_labels(dilated_nuclei_masks, name=f"{filename}_nuclei_mask_dilated")
    viewer.add_labels(eroded_nuclei_masks, name=f"{filename}_nuclei_mask_split")

    # Set a threshold value for the pixels in microglia channel
    threshold = 20  # Based on the microglia intensity

    # Create a new array with the same shape as nuclei_masks, initialized with False
    result_array = np.full_like(nuclei_masks, False, dtype=bool)

    # Find indices where values in values_array are above the threshold
    above_threshold_indices = microglia_mip > threshold

    # Update the corresponding positions in the result_array based on the mask_array
    nuclei_masks_bool = nuclei_masks.astype(bool) # Make a boolean copy of nuclei_masks to be able to use logical operators
    result_array[nuclei_masks_bool & above_threshold_indices] = True

    # Convert the boolean array to a binary array
    binary_result_array = result_array.astype(int)

    # Now, result_array contains True where both conditions are satisfied, and False otherwise
    #viewer.add_labels(binary_result_array, name=f"{filename}_nuclei+glia_coloc")

    # Erode binary_result_array to get rid of small nuclei pixels colocalizing with glia branches
    # Set the structuring element for erosion
    structuring_element = np.ones((5, 5), dtype=bool)  # You can adjust the size and shape

    # Perform binary erosion
    eroded_array = binary_erosion(binary_result_array, structure=structuring_element)
    #viewer.add_labels(eroded_array, name=f"{filename}_nuclei+glia_coloc_pe")

    # Now I want to recover just the nuclei_masks that are sitting on top of binary_results_array
    labels, num_labels = measure.label(nuclei_masks, return_num=True)

    # Create an array of indices corresponding to the True values in binary_result_array
    true_indices = np.where(eroded_array)

    # Initialize an array to store labels for each processed region
    processed_region_labels = np.zeros_like(nuclei_masks, dtype=int)

    # Label index for processed regions
    label_index = 1

    # Iterate over each connected component
    for label_ in range(1, num_labels + 1):
        # Extract the region corresponding to the current label
        region = (labels == label_)

        # Check if any True value in binary_result_array is present in the region
        if np.any(region[true_indices]):
            # Assign a unique label to the processed region
            processed_region_labels[region] = label_index
            label_index += 1

    #viewer.add_labels(processed_regions, name=f"{filename}_glia+_nuclei_mask")

    # Dilate processed_regions to make sure the dna_damage_foci objects sit inside
    dilated_glia_pos_nuclei = cle.dilate_labels(processed_region_labels, radius=3)
    # Erode processed_regions to obtain separate objects upon binarization
    eroded_glia_pos_nuclei = cle.erode_labels(processed_region_labels, radius=1)

    viewer.add_labels(processed_region_labels, name=f"{filename}_glia+_nuclei_mask")
    viewer.add_labels(dilated_glia_pos_nuclei, name=f"{filename}_glia+_nuclei_mask_dilated")
    viewer.add_labels(eroded_glia_pos_nuclei, name=f"{filename}_glia+_nuclei_mask_split")

    # Apply object segmenter from APOC
    segmenter = ObjectSegmenter(opencl_filename="./ObjectSegmenter.cl")
    dna_damage_masks = segmenter.predict(image=dna_damage_mip)

    # Erode dna_damage_masks to get rid of small artifacts
    # Set the structuring element for erosion
    structuring_element_damage = np.ones((2, 2), dtype=bool)  # You can adjust the size and shape

    # Perform binary erosion
    eroded_dna_damage = binary_erosion(dna_damage_masks, structure=structuring_element_damage)

    # Perform binary dilation to fill holes
    dilated_dna_damage = binary_dilation(eroded_dna_damage, structure=structuring_element_damage)

    #viewer.add_image(dna_damage_mip, name=f"{filename}_dna_damage_mip")
    #viewer.add_labels(dna_damage_masks, name=f"{filename}_dna_damage_foci")
    #viewer.add_labels(eroded_dna_damage, name=f"{filename}_dna_damage_foci_eroded")
    viewer.add_labels(dilated_dna_damage, name=f"{filename}_dna_damage_foci_dilated")
    
    # Label the DNA damage particles
    labeled_dna_damage, num_dna_damage = label(dilated_dna_damage)

    # Label the nuclei in eroded_glia_pos_nuclei and eroded_nuclei_masks
    labeled_glia_nuclei, num_glia_nuclei = label(eroded_glia_pos_nuclei)
    labeled_nuclei, num_nuclei = label(eroded_nuclei_masks)

    # Count particles in glia nuclei
    particles_in_glia_nuclei = count_particles_in_nuclei(labeled_dna_damage, labeled_glia_nuclei, num_glia_nuclei)

    # Count particles in other nuclei
    particles_in_nuclei = count_particles_in_nuclei(labeled_dna_damage, labeled_nuclei, num_nuclei)
    
    # Create a dictionary containing all extracted info per image
    stats_dict = {
        'filename': filename,
        'avg_dna_damage_foci/glia_+': np.mean(particles_in_glia_nuclei),
        'avg_dna_damage_foci/all_nuclei': np.mean(particles_in_nuclei),
        'nr_+_dna_damage_glia_nuclei': np.count_nonzero(particles_in_glia_nuclei),
        'nr_-_dna_damage_glia_nuclei': (particles_in_glia_nuclei == 0).sum(),
        'nr_glia_+_nuclei': processed_region_labels.max(),
        'nr_total_nuclei': nuclei_masks.max()}
    
    stats.append(stats_dict)
    
df = pd.DataFrame(stats)

df

,filename,avg_dna_damage_foci/glia_+,avg_dna_damage_foci/all_nuclei,nr_+_dna_damage_glia_nuclei,nr_-_dna_damage_glia_nuclei,nr_glia_+_nuclei,nr_total_nuclei
0,DSB Iba1 20_40X_CA1_test,0.666667,0.723077,3,3,6,130
1,DSB Iba1 20_40X_DG_test,1.125000,1.519031,6,2,8,289
2,DSB Iba1 21_40X_DG_test2,1.333333,1.085616,5,1,6,291
3,DSB Iba1 33_40X_CA3,0.857143,0.970238,4,3,7,168
4,DSB Iba1 33_40X_DG,1.909091,1.590504,7,4,11,337
5,DSB Iba1 43_40X_CTX2,0.200000,0.443114,3,17,20,167
6,DSB Iba1 45_40X_CTX3,0.523810,0.502762,7,14,21,181
7,DSB Iba1 45_40X_DG,1.045455,2.140984,14,8,22,305
8,DSB Iba1 58_40X_CTX2,1.111111,0.861702,12,6,18,188
9,DSB Iba1 58_40X_CTX3,1.800000,0.622642,4,1,5,159


Dev below, working version but I need to extract DNA foci numbers

TODO: Extract DNA DAMAGE foci NUMBERS

In [ ]:
viewer = napari.Viewer(ndisplay=2)
model = models.Cellpose(gpu=True, model_type='nuclei')

image = images[2]

# Extract filename
file_path = Path(image)
filename = file_path.stem

# Read the image file 
img = tifffile.imread(image)

# Extract the stack containing the nuclei (0), dna_damage (1) and microglia channel (2)
nuclei_stack = img[:,0,:,:]
dna_damage_stack = img[:,1,:,:]
microglia_stack = img[:,2,:,:]

# Perform maximum intensity projections
nuclei_mip = np.max(nuclei_stack, axis=0)
dna_damage_mip = np.max(dna_damage_stack, axis=0)
microglia_mip = np.max(microglia_stack, axis=0)

# Stack the arrays along a new axis (axis 0)
stack = np.stack((nuclei_mip, dna_damage_mip, microglia_mip), axis=0)

# Create a copy of nuclei_mip
input_img = nuclei_mip.copy()

# Might need to perform a Gaussian-blur before
post_gaussian_img = skimage.filters.gaussian(input_img, sigma=6)

# Predict nuclei nuclei_masks using cellpose
nuclei_masks, flows, styles, diams = model.eval(post_gaussian_img, diameter=None, channels=[0, 0], net_avg=False)

# Dilate nuclei to make sure the dna_damage_foci objects sit inside
dilated_nuclei_masks = cle.dilate_labels(nuclei_masks, radius=3)
# Erode dilated_nuclei_masks to obtain separate objects upon binarization
eroded_nuclei_masks = cle.erode_labels(dilated_nuclei_masks, radius=1)

# Show the resulting stack in Napari
viewer.add_image(stack, name=f"{filename}_mip")
#viewer.add_image(nuclei_mip, name=f"{filename}_nuclei")
viewer.add_labels(nuclei_masks, name=f"{filename}_nuclei_mask")
viewer.add_labels(dilated_nuclei_masks, name=f"{filename}_nuclei_mask_dilated")
viewer.add_labels(eroded_nuclei_masks, name=f"{filename}_nuclei_mask_split")

# Set a threshold value for the pixels in microglia channel
threshold = 20  # Based on the microglia intensity

# Create a new array with the same shape as nuclei_masks, initialized with False
result_array = np.full_like(nuclei_masks, False, dtype=bool)

# Find indices where values in values_array are above the threshold
above_threshold_indices = microglia_mip > threshold

# Update the corresponding positions in the result_array based on the mask_array
nuclei_masks_bool = nuclei_masks.astype(bool) # Make a boolean copy of nuclei_masks to be able to use logical operators
result_array[nuclei_masks_bool & above_threshold_indices] = True

# Convert the boolean array to a binary array
binary_result_array = result_array.astype(int)

# Now, result_array contains True where both conditions are satisfied, and False otherwise
#viewer.add_labels(binary_result_array, name=f"{filename}_nuclei+glia_coloc")

# Erode binary_result_array to get rid of small nuclei pixels colocalizing with glia branches
# Set the structuring element for erosion
structuring_element = np.ones((5, 5), dtype=bool)  # You can adjust the size and shape

# Perform binary erosion
eroded_array = binary_erosion(binary_result_array, structure=structuring_element)
#viewer.add_labels(eroded_array, name=f"{filename}_nuclei+glia_coloc_pe")

# Now I want to recover just the nuclei_masks that are sitting on top of binary_results_array
labels, num_labels = measure.label(nuclei_masks, return_num=True)

# Create an array of indices corresponding to the True values in binary_result_array
true_indices = np.where(eroded_array)

# Initialize an array to store labels for each processed region
processed_region_labels = np.zeros_like(nuclei_masks, dtype=int)

# Label index for processed regions
label_index = 1

# Iterate over each connected component
for label in range(1, num_labels + 1):
    # Extract the region corresponding to the current label
    region = (labels == label)

    # Check if any True value in binary_result_array is present in the region
    if np.any(region[true_indices]):
        # Assign a unique label to the processed region
        processed_region_labels[region] = label_index
        label_index += 1

#viewer.add_labels(processed_regions, name=f"{filename}_glia+_nuclei_mask")

# Dilate processed_regions to make sure the dna_damage_foci objects sit inside
dilated_glia_pos_nuclei = cle.dilate_labels(processed_region_labels, radius=3)
# Erode processed_regions to obtain separate objects upon binarization
eroded_glia_pos_nuclei = cle.erode_labels(processed_region_labels, radius=1)

viewer.add_labels(processed_region_labels, name=f"{filename}_glia+_nuclei_mask")
viewer.add_labels(dilated_glia_pos_nuclei, name=f"{filename}_glia+_nuclei_mask_dilated")
viewer.add_labels(eroded_glia_pos_nuclei, name=f"{filename}_glia+_nuclei_mask_split")

# Apply object segmenter from APOC
segmenter = ObjectSegmenter(opencl_filename="./ObjectSegmenter.cl")
dna_damage_masks = segmenter.predict(image=dna_damage_mip)

# Erode dna_damage_masks to get rid of small artifacts
# Set the structuring element for erosion
structuring_element_damage = np.ones((2, 2), dtype=bool)  # You can adjust the size and shape

# Perform binary erosion
eroded_dna_damage = binary_erosion(dna_damage_masks, structure=structuring_element_damage)

# Perform binary dilation to fill holes
dilated_dna_damage = binary_dilation(eroded_dna_damage, structure=structuring_element_damage)

#viewer.add_image(dna_damage_mip, name=f"{filename}_dna_damage_mip")
#viewer.add_labels(dna_damage_masks, name=f"{filename}_dna_damage_foci")
#viewer.add_labels(eroded_dna_damage, name=f"{filename}_dna_damage_foci_eroded")
viewer.add_labels(dilated_dna_damage, name=f"{filename}_dna_damage_foci_dilated")




In [ ]:
import numpy as np
from scipy.ndimage import label

# Assuming dilated_dna_damage, dilated_glia_pos_nuclei, and dilated_nuclei_masks are already defined in your loop

# Label the DNA damage particles
labeled_dna_damage, num_dna_damage = label(dilated_dna_damage)

# Label the nuclei in eroded_glia_pos_nuclei and eroded_nuclei_masks
labeled_glia_nuclei, num_glia_nuclei = label(eroded_glia_pos_nuclei)
labeled_nuclei, num_nuclei = label(eroded_nuclei_masks)

# Function to count particles in nuclei
def count_particles_in_nuclei(labeled_particles, labeled_nuclei, num_nuclei):
    particle_counts = np.zeros(num_nuclei, dtype=int)
    for i in range(1, num_nuclei + 1):
        nuclei_mask = labeled_nuclei == i
        # Count the number of unique particles in the nucleus
        unique_particles = np.unique(labeled_particles[nuclei_mask])
        # Exclude the background label (0)
        particle_counts[i - 1] = len(unique_particles[unique_particles > 0])
    return particle_counts

# Count particles in glia nuclei
particles_in_glia_nuclei = count_particles_in_nuclei(labeled_dna_damage, labeled_glia_nuclei, num_glia_nuclei)

# Count particles in other nuclei
particles_in_nuclei = count_particles_in_nuclei(labeled_dna_damage, labeled_nuclei, num_nuclei)

# The arrays particles_in_glia_nuclei and particles_in_nuclei now contain the counts of DNA damage particles
# in each nucleus of dilated_glia_pos_nuclei and dilated_nuclei_masks respectively

In [ ]:
particles_in_glia_nuclei

In [ ]:
particles_in_nuclei

In [ ]:
# Create a dictionary containing all extracted info per image
stats = [
    {
        'filename': filename,
        'avg_dna_damage_foci/glia_+': np.mean(particles_in_glia_nuclei),
        'avg_dna_damage_foci/all_nuclei': np.mean(particles_in_nuclei),
        'nr_+_dna_damage_glia_nuclei': np.count_nonzero(particles_in_glia_nuclei),
        'nr_-_dna_damage_glia_nuclei': (particles_in_glia_nuclei == 0).sum(),
        'nr_glia_+_nuclei': processed_region_labels.max(),
        'nr_total_nuclei': nuclei_masks.max()
    }
]

In [ ]:
stats

Detecting DNA damage foci in dna_damage_mip



In [ ]:
import matplotlib.pyplot as plt

# Visualize the original image and segmentation nuclei_masks
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(input_img, cmap='gray')
plt.title('Original Image')

plt.subplot(1, 2, 2)
plt.imshow(nuclei_masks, cmap='viridis')
plt.title('Segmentation nuclei_masks')

plt.show()